# Hypothesis testing problems

## Exercise 1

You are a nutritionist researching two different types of diets to see if there is a significant difference in weight loss after one month. You choose two random groups of people; one group follows the first diet, and the other follows the second. At the end of the month, you record the weight loss (in kg) for each person:

| Diet 1 | Diet 2 |
|:-------|:-------|
| 2.0 | 3.0 |
| 2.5 | 3.2 |
| 3.0 | 3.1 |
| 2.8 | 2.9 |
| 2.3 | 2.8 |
| 2.7 | 3.0 |
| 2.5 | 3.2 |

With these data, it seeks to answer the following question: Is there a significant difference in average weight loss between people who followed the first diet and those who followed the second diet?

To draw conclusions, follow the points below:

- State the hypothesis: null and alternative hypothesis.
- Perform the test to test the hypothesis. You can use a Student's t-test.
- Analyze the conclusions.

In [1]:
# Null: mu(Diet 1) = mu(Diet 2)
# Alternative: mu(Diet 1) != mu(Diet 2)
#

import numpy as np
from math import e
from scipy.stats import ttest_ind

def student_t(s1, s2):
    def mean(sample):
        return(sum(sample)/len(sample))
    def std(sample):
        dev = [(i - mean(sample))**2 for i in sample]
        return sum(dev)/(len(dev) - 1)
    def cdf_finder(t_stat):
        x = np.sqrt(2) * np.log((2 * abs(t_stat)) - 1)
        return x
    x1, x2, d1, d2, n1, n2 = mean(s1), mean(s2), std(s1), std(s2), len(s1), len(s2)
    ts, df = (x1 - x2)/np.sqrt((d1/n1) + (d2/n2)), n1+n2-2
    result = [ts, df, (2 * (1 - cdf_finder(ts))), ttest_ind(s1, s2)[1]]
    return result

diet1 = [2.0,2.5,3.0,2.8,2.3,2.7,2.5]
diet2 = [3.0, 3.2, 3.1, 2.9, 2.8, 3.0, 3.2]

print(student_t(diet1, diet2))

# H0 can be discarded, and H1 is True


[-3.5383407969933938, 12, -3.1037800924200045, 0.004083270191713912]


## ANOVA

**ANOVA** (*Analysis of Variance*) is a statistical technique used to compare the measures of two or more groups. The idea behind ANOVA is to decompose the total variability in the data into two components: between-group variability and within-group variability:

- **Between-group variability**: This variability refers to the differences between the group means. If this variability is considerably larger than the within-group variability, it could be an indication that at least one of the group means is different.
- **Within-group variability**: This variability refers to the dispersion of the data within each group. If all groups have similar variability, then any noticeable difference in group means could be considered significant.

Hypotheses in ANOVA typically include:

- **Null hypothesis** ($H₀$): The means of all groups are equal.
- **Alternative hypothesis** ($H₁$): At least one of the group means is different.

If the ANOVA test result is significant (e.g., a p-value less than a threshold such as 0.05), this suggests that at least one group mean is different.

## Exercise 2

A farmer decides to test three different types of fertilizers to determine if one is superior in terms of corn production. The farmer plants corn on 15 identical plots and uses all three fertilizers (5 plots for each type). At the end of the season, he measures the corn yield (in kg) of each plot, with the following result:

| Fertilizer 1 | Fertilizer 2 | Fertilizer 3 |
|:-------------|:-------------|:-------------|
| 20 | 22 | 24 |
| 21 | 21 | 23 |
| 20 | 23 | 22 |
| 19 | 22 | 23 |
| 20 | 21 | 24 |

With this data, he seeks to answer the following question: Is there a significant difference in average corn yield between the three types of fertilizers?

To help you, follow the points below:

- State the hypothesis: null and alternative hypothesis.
- Perform the ANOVA test.
- Analyze the conclusions.
- If one fertilizer is better than another, how can we know it?

In [2]:
# H0: mu(F1) = mu(F2) = mu(F3)
# H1: mu(F1) != mu(F2) != mu(F3)

from scipy.stats import f_oneway
import pandas as pd

def anova_man(ds, daf = False):
    base = len(ds) + 1
    suma = 0
    lena = 0
    datf = pd.DataFrame({'Data':[], 'Mean':[], 'Length':[], 'DF':[], 'SSE':[], 'SSR':[], 'F':[]})
    for i in range(len(ds)):
        suma += sum(ds[i])
        lena += len(ds[i])
        datf.loc[i, 'Data'] = str(ds[i])
        datf.loc[i, 'Mean'] = sum(ds[i])/len(ds[i])
        datf.loc[i, 'Length'] = len(ds[i])
        datf.loc[i, 'DF'] = len(ds[i]) - 1
    meana = suma/lena
    def mean(sample):
        return(sum(sample)/len(sample))
    def sse_quick(dataset):
        sse = 0
        for i in range(len(dataset)):
            sse_p = sse
            m = mean(dataset[i])
            for j in dataset[i]:
                sse += (j-m)**2
            datf.loc[i, 'SSE'] = sse - sse_p
        datf.loc[base, 'SSE'] = sse
        return sse
    def ssr_quick(dataset):
        ssr = 0
        for i in range(len(dataset)):
            m = mean(dataset[i])
            r = len(dataset[i]) * ((m - meana)**2)
            ssr += r
            datf.loc[i, 'SSR'] = r
        datf.loc[len(ds)+1, 'SSR'] = ssr
        return ssr
    ssr_df = len(ds) - 1
    sse_df = lena - len(ds)
    F_value = (ssr_quick(ds)/ssr_df)/(sse_quick(ds)/sse_df)
    datf.loc[base, 'DF'] = f'SSR:{ssr_df}, SSE:{sse_df}'
    datf.loc[base, 'F'] = F_value
    if not daf:
        return [F_value, ssr_df, sse_df, f_oneway(*ds)[1]]
    else:
        return datf

F1 = [20, 21, 20, 19, 20]
F2 = [22, 21, 23, 22, 21]
F3 = [24, 23, 22, 23, 24]
ds = [F1, F2, F3]
print(anova_man(ds))
print(anova_man(ds, daf=True))
# Based on the p-value, there is statistically significant differences between the three fertilizers

ds2 = F1 + F2 + F3
labels = ['F1'] * 5 + ['F2'] * 5 + ['F3'] * 5

from statsmodels.stats.multicomp import pairwise_tukeyhsd
print(pairwise_tukeyhsd(ds2, labels))

# Based on the rejection and p scores of all comparisons, it is safe to say that none of the means resemble eachother,
# just as it is equally clear from the meandiff that F3 is the strongest fertilizer based on available data

/tmp/ipykernel_5623/269886319.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[20, 21, 20, 19, 20]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  datf.loc[i, 'Data'] = str(ds[i])
/tmp/ipykernel_5623/269886319.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'SSR:2, SSE:12' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  datf.loc[base, 'DF'] = f'SSR:{ssr_df}, SSE:{sse_df}'
/tmp/ipykernel_5623/269886319.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[20, 21, 20, 19, 20]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  datf.loc[i, 'Data'] = str(ds[i])
/tmp/ipykernel_5623/269886319.py:44: FutureWarning:

[20.315789473684198, 2, 12, 0.00014047824793190475]
                   Data  Mean  Length             DF  SSE        SSR  \
0  [20, 21, 20, 19, 20]  20.0     5.0            4.0  2.0  13.888889   
1  [22, 21, 23, 22, 21]  21.8     5.0            4.0  2.8   0.088889   
2  [24, 23, 22, 23, 24]  23.2     5.0            4.0  2.8  11.755556   
4                   NaN   NaN     NaN  SSR:2, SSE:12  7.6  25.733333   

           F  
0        NaN  
1        NaN  
2        NaN  
4  20.315789  
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
    F1     F2      1.8 0.0099 0.4572 3.1428   True
    F1     F3      3.2 0.0001 1.8572 4.5428   True
    F2     F3      1.4 0.0409 0.0572 2.7428   True
--------------------------------------------------
